<center>
<img src="https://2223moodle.isel.pt/pluginfile.php/1/theme_adaptable/logo/1631635830/logo-isel_inv3.png" width="250">
<h3>Licenciatura em Engenharia Informática e Multimédia</h3><br>
<br>
<h2>Processamento de Imagem e Visão (PIV)</h3>
<h3>2º Trabalho Laboratorial – Estimação e classificação de movimento </h3>
<br>

<br>
Trabalho Realizado por:<br>
Gonçalo Silva <b>A48328</b><br>
Diogo Lobo <b>A48168</b><br>
Turma 52D<br><br>
Docente: João Pedro Costa <br>
<br>
27 de Novembro 2024
</center>


In [1]:
# Imports
import cv2
import numpy as np
import time
import pygame
from pygame.locals import QUIT
from pygame import surfarray


pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Inicialização dos tamanhos da janela e objetos
window_width = 1280
window_height = 700
cube_size = 50
cube_x = window_width // 4 - cube_size
cube_y = window_height // 2 - cube_size
cube_color = (0, 0, 255)
previousContour = None
radius = 20
cyNew = None
cxNew = None
cxOld = None
cyOld = None
oldArea = None
newArea = None

## Final 

In [3]:
import cv2
import numpy as np
import pygame

# Inicializa a câmera e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack")
fonte = pygame.font.SysFont("Arial", 64)

running = True

while running:
    ret, frame = cap.read()
    if not ret:
        break

    # Converte a imagem para o espaço HSV e escala de cinza
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Ajuste conforme necessário
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    
    # mask = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=2)
    # mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2)

    # Encontra contornos da máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Inicializa flag para verificar se a mão foi detectada
    hand_detected = False

    if contours:
        # Seleciona o maior contorno, assumindo que é a mão
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 5000:  # Filtra áreas pequenas (ajustar conforme necessário)
            hand_detected = True

            # Desenha o contorno da mão
            cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)

            # Cria uma máscara apenas para a mão detectada
            hand_mask = np.zeros_like(mask)
            cv2.drawContours(hand_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

            # Aplica a máscara da mão à imagem em escala de cinza
            hand_region = cv2.bitwise_and(gray, gray, mask=hand_mask)

            # Detecta características usando goodFeaturesToTrack apenas na região da mão
            corners = cv2.goodFeaturesToTrack(hand_region, maxCorners=50, qualityLevel=0.01, minDistance=10)
            if corners is not None:
                for corner in corners:
                    x, y = corner.ravel()
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)  # Desenha círculos nos pontos detectados

    ##### Vizualização da câmera no Pygame e sua colocação #####

    # Converte de BGR (OpenCV) para RGB (Pygame)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.rot90(frame)  # Pygame espera a imagem transposta
    pygame_surface = pygame.surfarray.make_surface(frame)
    window.fill((0, 0, 0))  # Limpa a tela antes de atualizar o frame
    window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela

    # Mostra o texto apenas se a mão for detectada
    if hand_detected:
        texto_surface = fonte.render("Hand Detected", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 520, 10))  # Posição do texto

    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()


# Final com Bolinha

In [4]:
def canMove(x,y,r,direction):
    if(direction == 0):
        return (y >= r)                 and ((y<= 250-r) or (x-r>=250) or (y>250+r))   and ((y<= 250-r) or (x+r<388) or (y>250+r))
    if(direction == 1):
        return (y <= window_height - r) and ((y<= 200-r) or (x-r>=250) or (y>200+r))   and ((y<= 200-r) or (x+r<388) or (y>200+r))
    if(direction == 2):
    # and ((x-r>=250) or (250<y<200)) 
        return (x >= r)   and ((x - r >= 250) or not (200 - r <= y <= 250 + r))  
    if(direction == 3):
        return (x <= window_width/2 - r) and ((x + r <= 388) or not (200 - r <= y <= 250 + r))  
    
def convImage(frame):
    # Converte a imagem para o espaço HSV e escala de cinza
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Ajuste conforme necessário
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    
    mask = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=2)
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2)

    # Encontra contornos da máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return gray,mask,contours


def moments(largest_contour):
        Mnew = cv2.moments(largest_contour)
        if Mnew['m00'] != 0:
            cxNew = int(Mnew['m10'] / Mnew['m00'])
            cyNew = int(Mnew['m01'] / Mnew['m00'])
            newArea = cv2.contourArea(largest_contour)
        else:
            cxNew, cyNew, newArea = 0, 0, 0  # Fallback values

        return cxNew,cyNew,newArea

In [5]:

# Inicializa a camara e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack")
fonte = pygame.font.SysFont("Arial", 64)
fonte2 = pygame.font.SysFont("Arial", 50)
running = True
player_pos = pygame.Vector2(window.get_width() // 4, window.get_height() // 2)
dt = 0
clock = pygame.time.Clock()
currentTime = time.time()

while running:
    if(currentTime>10):
        window.fill((0, 0, 0))
        
        ret, frame = cap.read()
        if not ret:
            break

        gray, mask, contours = convImage(frame)

        # Inicializa flag para verificar se a mão foi detectada
        hand_detected = False

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 5000:  # Ensure a valid hand contour
                if previousContour is None:
                    previousContour = largest_contour.copy()

                hand_detected = True

                cxNew,cyNew,newArea = moments(largest_contour)

                cxOld, cyOld, oldArea = moments(previousContour)
                
                # Desenha o contorno da mão
                cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)

                # Cria uma máscara apenas para a mão detectada
                hand_mask = np.zeros_like(mask)
                cv2.drawContours(hand_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

                # Aplica a máscara da mão à imagem em escala de cinza
                hand_region = cv2.bitwise_and(gray, gray, mask=hand_mask)

                # Detecta características usando goodFeaturesToTrack apenas na região da mão
                corners = cv2.goodFeaturesToTrack(hand_region, maxCorners=50, qualityLevel=0.01, minDistance=10)
                if corners is not None:
                    for corner in corners:
                        x, y = corner.ravel()
                        cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)  # Desenha círculos nos pontos detectados

        ##### Vizualização da camara no Pygame e sua colocação #####

        # Converte de BGR (OpenCV) para RGB (Pygame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = np.rot90(frame)  # Pygame espera a imagem transposta
        pygame_surface = pygame.surfarray.make_surface(frame)
        window.fill((0, 0, 0))  # Limpa a tela antes de atualizar o frame
        window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela
        pygame.draw.rect(window,(255,255,255),(0,200,250,50))
        pygame.draw.rect(window,(255,255,255),(388,200,250,50))
        # Mostra o texto apenas se a mão for detectada
        if hand_detected:
            texto_surface = fonte.render("Hand Detected", True, (255, 0, 0))  # Texto em vermelho
            window.blit(texto_surface, (window_width - 520, 10))  # Posição do texto
        
    
        pygame.draw.circle(window, "red", player_pos, radius)
        keys = pygame.key.get_pressed()
        if keys[pygame.K_w] or (cyNew is not None and cyOld is not None and cyNew < cyOld + 5 and canMove(player_pos.x,player_pos.y, radius, 0)):
            # clockUp = time.time()
            # while time.time()-clockUp<.01:
            pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
            player_pos.y -= 1000 * dt
            texto_surface = fonte2.render("Up", True, (255, 0, 0))  # Texto em vermelho
            window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
                # pygame.display.update()
                # dt = clock.tick(60) / 1000
        if keys[pygame.K_s] or (cyNew is not None and cyOld is not None and cyNew + 5 > cyOld and canMove(player_pos.x,player_pos.y, radius, 1)):
            # clockDown = time.time()
            # while time.time()-clockDown<0.01:
            pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
            player_pos.y += 1000 * dt
            texto_surface = fonte2.render("Down", True, (255, 0, 0))  # Texto em vermelho
            window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
                # pygame.display.update()
                # dt = clock.tick(60) / 1000
        if keys[pygame.K_a] or (cxNew is not None and cxOld is not None and cxNew + 5 > cxOld and canMove(player_pos.x,player_pos.y, radius, 2)):
            # clockLeft = time.time()
            # while time.time()-clockLeft<.01:
            pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
            player_pos.x -= 1000 * dt
            texto_surface = fonte2.render("Left", True, (255, 0, 0))  # Texto em vermelho
            window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
                # pygame.display.update()
                # dt = clock.tick(60) / 1000
        if keys[pygame.K_d] or (cxNew is not None and cxOld is not None and cxNew < cxOld + 5 and canMove(player_pos.x,player_pos.y, radius, 3)):
            # clockRight = time.time()
            # while time.time()-clockRight<.01:
            pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
            player_pos.x += 1000 * dt
            texto_surface = fonte2.render("Right", True, (255, 0, 0))  # Texto em vermelho
            window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
                # pygame.display.update()
                # dt = clock.tick(60) / 1000
        if keys[pygame.K_PLUS] or (newArea is not None and oldArea is not None and newArea + 750 > oldArea):
            pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
            if(radius<1000):
                radius += 100 * dt
            texto_surface = fonte2.render("Zoom In", True, (255, 0, 0))  # Texto em vermelho
            window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto

        if keys[pygame.K_MINUS] or (newArea is not None and oldArea is not None and newArea < oldArea + 750):
            pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
            if(radius>10):
                radius -= 100 * dt
            texto_surface = fonte2.render("Zoom Out", True, (255, 0, 0))  # Texto em vermelho
            window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto


        texto_surface = fonte.render("Movement:", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 600, window_height - 100))  # Posição do texto
        previousContour = largest_contour.copy()
        

    pygame.display.update()
    dt = clock.tick(60) / 1000
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()

## Teste 1

In [6]:
#Inicialização da Camera 
camera = 0
cam = cv2.VideoCapture(camera)

running = True


pygame.init()

window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Webcam in Pygame")

running = True
fonte = pygame.font.SysFont("Arial", 32)
previousFrame = None
thr = 10
kernelSize = 9
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernelSize,kernelSize),(-1,-1))

if cam.isOpened():
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        flag, frame = cam.read()
        if flag:

            frameE = cv2.medianBlur(frame,kernelSize)

            if previousFrame is None:
                previousFrame = frame

            diff_image = cv2.absdiff(frame,previousFrame)
            _ ,bw1 = cv2.threshold(diff_image,thr,255,cv2.THRESH_BINARY)
            _, bw2 = cv2.threshold(np.uint8(bw1.sum(axis=2)),1,255,cv2.THRESH_BINARY)

            erode_image = cv2.erode(bw2,kernel)
            dilate_image = cv2.dilate(erode_image,kernel,iterations=2)

            (num_labels,label_image,stats,centroids) = cv2.connectedComponentsWithStats(dilate_image)

            previousFrame = frame.copy()

            motionFlag = False

            for i in range(1,num_labels):
                if(stats[i,4]>0.05*bw2.shape[0]*bw2.shape[1]):
                    cv2.rectangle(frame,(stats[i,0],stats[i,1]),(stats[i,0]+stats[i,2],stats[i,1]+stats[i,3]),(0,0,255),thickness=2)
                    motionFlag = True

            if motionFlag:
                texto_surface = fonte.render("Motion Detected", True, (255, 0, 0))  # Texto em vermelho
                window.blit(texto_surface, (10, 10))  # Posição do texto

            ##### Vizualização da camera no Pygame e sua colocação ######

            # Converte de BGR (OpenCV) para RGB (Pygame)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = np.rot90(frame)  # Pygame espera a imagem transposta
            pygame_surface = pygame.surfarray.make_surface(frame)
            window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela

        pygame.display.update()  # Atualiza a tela

    # Libera recursos ao sair
    cam.release()
    pygame.quit()

else:
    print("Error: Could not open Camera")


## Teste 2 

In [7]:
import cv2
import numpy as np
import pygame

# Inicializa a câmera e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack1")

running = True

while running:
    ret, frame = cap.read()
    if not ret:
        break

    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele (opcional)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Aplica a máscara e detecta as características
    masked_gray = cv2.bitwise_and(gray, gray, mask=mask)

    # Detecta características usando goodFeaturesToTrack
    corners = cv2.goodFeaturesToTrack(masked_gray, maxCorners=50, qualityLevel=0.01, minDistance=10)
    if corners is not None:
        for corner in corners:
            x, y = corner.ravel()
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)  # Desenha círculos nos pontos detectados

    # Exibe a imagem com os pontos detectados no Pygame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = np.rot90(frame_rgb)
    pygame_surface = pygame.surfarray.make_surface(frame_rgb)
    window.blit(pygame_surface, (0, 0))
    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()
